In [1]:
import loconsensus.loconsensus as loconsensus
import locomotif.visualize as visualize
import matplotlib.pyplot as plt
import pickle
import numpy as np
from pathlib import Path
from dtaidistance import dtw_ndim

s_dir = Path('../pickles/subjects.pkl')
with open(s_dir, 'rb') as f:
    subjects = pickle.load(f)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Code\Repos\loconsensus\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Code\Repos\loconsensus\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Code\Repos\loconsensus\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Code\Repos\loc

In [2]:
sampling_frequency = 10
ts_names = [(1, 'walking'),     (1, 'running'),
            (2, 'walking'),     (2, 'running'),
            (5, 'walking'),     (5, 'running'),
            (6, 'running'),     (6, 'walking'),
            (8, 'walking'),     (8, 'running'),
            (1, 'cycling'),     (1, 'desc_stairs'),
            (2, 'desc_stairs'), (2, 'cycling'),
            (4, 'cycling'),     (4, 'desc_stairs'),
            (5, 'desc_stairs'), (5, 'cycling'),
            (6, 'cycling'),     (6, 'desc_stairs'),
            (7, 'desc_stairs'), (7, 'cycling'),
            (8, 'cycling'),     (8, 'desc_stairs'),
            (1, 'rope_jumping'),(1, 'ironing'),
            (2, 'ironing'),     (2, 'rope_jumping'),
            (5, 'rope_jumping'),(5, 'ironing'),
            (8, 'ironing'),     (8, 'rope_jumping')]
gt = [0,0,0,0,0,
      1,1,1,1,1,1,1,
      2,2,2,2
      ]
assert len(ts_names) == len(gt) * 2
ts_list = [subjects.get(f'subject10{subject}').get(activity) for (subject, activity) in ts_names]
ts_list = [ts[:, 6:] for ts in ts_list]


In [3]:
it = iter(ts_list)
ts_list = [np.concatenate((ts1, ts2)) for ts1, ts2 in zip(it,it)]
assert len(ts_list) == len(gt)

for ts in ts_list:
    print(ts.shape)

(4353, 3)
(4178, 3)
(5669, 3)
(4856, 3)
(4808, 3)
(3100, 3)
(3254, 3)
(2966, 3)
(3033, 3)
(2559, 3)
(2801, 3)
(3029, 3)
(3650, 3)
(4215, 3)
(4078, 3)
(4180, 3)


In [4]:
ds = dtw_ndim.distance_matrix(ts_list, use_c=True)
ds = (ds - ds.min()) / (ds.max() - ds.min())
print(ds)

[[0.         0.59802538 0.72090242 0.81538686 0.93364872 0.84256428
  0.82533453 0.84202378 0.86280363 0.83970667 0.82395814 0.87426951
  0.88609818 0.86286112 0.88557596 0.91049626]
 [0.59802538 0.         0.76777262 0.85057848 0.87652292 0.82483648
  0.76830872 0.85073508 0.85834535 0.81591088 0.81312361 0.78632403
  0.81872882 0.78041171 0.81137385 0.84628082]
 [0.72090242 0.76777262 0.         0.89549144 0.94155408 0.95392108
  0.94924741 0.92729524 0.94352873 0.93798157 0.9367833  0.96694685
  0.97805014 0.96601224 0.97435889 0.98216363]
 [0.81538686 0.85057848 0.89549144 0.         1.         0.89093456
  0.87832898 0.87637494 0.88361988 0.89213256 0.84843974 0.98304616
  0.88924173 0.91662726 0.90916322 0.95829075]
 [0.93364872 0.87652292 0.94155408 1.         0.         0.90054319
  0.84131497 0.89917148 0.92845704 0.87096787 0.89183521 0.8312352
  0.93470981 0.84186593 0.91119498 0.86172368]
 [0.84256428 0.82483648 0.95392108 0.89093456 0.90054319 0.
  0.62354279 0.52613022 0.

In [5]:
global_offsets = np.cumsum([0] + [len(ts) for ts in ts_list])
l_min = 15
l_max = 30
rho = 0.8

nb = None

In [ ]:
#ms = loconsensus.apply_loconsensus(ts_list, l_min, l_max, rho, nb, overlap=0.5)
with open('../pickles/cm_double.pkl', 'rb') as handle:
    ms = pickle.load(handle)

EOFError: Ran out of input

In [ ]:
n = len(ts_list)

cm = []
for _, mset, _, bf in ms:
    cm.append((mset, bf))

cms = []
for mset, bf in cm:
    m = {i: [] for i in range(len(ts_list))}
    for (s, e) in mset:
        i = np.sum(global_offsets <= s) - 1
        m[i].append(((s - global_offsets[i], e - global_offsets[i]), bf))
    cms.append(m)

S = np.zeros((n, n))

for cs in cms:
    for i in range(n):
        for j in range(n):
            cs1 = cs[i]
            cs2 = cs[j]
            mbf = 0
            for c in cs1:
                smbf = c[1]
                if smbf > mbf:
                    mbf = smbf
            for c in cs2:
                smbf = c[1]
                if smbf > mbf:
                    mbf = smbf
            if mbf == 0:
                continue
            # print(f'{mbf} // {len(cs1)} {len(cs2)}')
            S[i, j] += min(len(cs1), len(cs2))
            #S[i, j] += mbf * min(len(cs1), len(cs2))
smin = S.min()
smax = S.max()
S = (S - smin) / (smax - smin)

D = 1 - S
D[np.diag_indices(len(D))] = 0
print(D.shape)

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform
import pandas as pd

ts_names = gt

Cs = []
Zs = []

# double = 3
n_clust = 3

for k in [n_clust]:
    spectral = SpectralClustering(n_clusters=k, affinity='precomputed')
    clusters = spectral.fit_predict(S)
    Cs.append(clusters)
    Z = linkage(squareform(D), method='single')
    h = fcluster(Z, k, criterion='maxclust')
    Zs.append(h)
    fig, ax = plt.subplots(figsize=(5, 5))
    dendrogram(Z, labels=ts_names, ax=ax, leaf_rotation=90)
    plt.show()

df = pd.DataFrame({'Label': ts_names})
for i, k in enumerate([n_clust]):
    df[f'Spectral {k}'] = Cs[i]
    df[f'Hierarchical {k}'] = Zs[i]

df.set_index('Label', inplace=True)
#print(df)

ari2 = adjusted_rand_score(gt, Cs[0])
ari3 = adjusted_rand_score(gt, Zs[0])
nmi2 = normalized_mutual_info_score(gt, Cs[0])
nmi3 = normalized_mutual_info_score(gt, Zs[0])
df_metrics = pd.DataFrame({
    'Method': ['Spectral Clustering', 'Hierarchical Clustering'],
    'ARI': [ari2, ari3],
    'NMI': [nmi2, nmi3]
})
print(df_metrics)

In [ ]:
from scipy.linalg import eigh
from scipy.sparse.csgraph import laplacian
L = laplacian(S, normed=True)

eigenvalues, eigenvectors = eigh(L)
#embedding = eigenvectors[:, 1:3]
embedding = eigenvectors[:, 1:3]

plt.figure(figsize=(4, 4))
for i in range(n_clust):
    plt.scatter(embedding[clusters == i, 0], embedding[clusters == i, 1], label=f'Cluster {i+1}')
plt.title('Spectral Embedding')
plt.xlabel('Eigenvector 1')
plt.ylabel('Eigenvector 2')
plt.legend()
plt.show()

In [ ]:
Z1 = linkage(squareform(ds), method='single')
h1 = fcluster(Z1, k, criterion='maxclust')
fig, ax = plt.subplots(figsize=(5, 5))
dendrogram(Z1, labels=ts_names, ax=ax, leaf_rotation=90)
plt.show()
ari4 = adjusted_rand_score(gt, h1)
nmi4 = normalized_mutual_info_score(gt, h1)
print(f'ari: {ari4} // nmi: {nmi4}')

In [ ]:
from sklearn.manifold import MDS

mds2 = MDS(n_components=2, dissimilarity='precomputed')
points2 = mds2.fit_transform(ds)

plt.figure(figsize=(4, 4))
scatter = plt.scatter(points2[:, 0], points2[:, 1], c=gt)
for i, label in enumerate(ts_names):
    #plt.annotate(label[1], (points2[i, 0], points2[i, 1]))
    plt.annotate(label, (points2[i, 0], points2[i, 1]))